# PRNG

## State and seed

 [python random module](https://docs.python.org/3/library/random.html).
 1. Execute few times to see the PRNG generates diffeent outputs.
 2. Initialize the generator using the seed=0 and execute the cell several times.
 3. Print out the internal state - use **getstate** method of random package. 

In [69]:
import random

random.randint(0, 10)

6

# PRNG generator
## LCG 
 1. Execute the cell with the simple generator **LCG** defined by the linear update function **lcg_update**. What is the problem with the generator?
 2. What is the maximum of values **LCG** can generate?  
 3. Find the best parameters a,c (maximal cycle).
  - What does it mean the best params? 

In [70]:
id = lambda x: x

class PRNG: 
    def __init__(self, state_size, init_f=id, update_f=id, output_f = id):
        self.state = b"\x00"*state_size
        self.init_f = init_f
        self.update_f = update_f
        self.output_f = output_f
        
    def seed(self, seed):
        self.state = self.init_f(seed)
        
    def random(self):       
        self.state = self.update_f(self.state)
        return self.output_f(self.state)
        
def lcg_update(x, a=0, c=0, m=2^31):
    return (a*x+c) % m


LCG = PRNG(4, init_f=id, update_f = lambda x: lcg_update(x, a=2, c=1, m=256), output_f=id)
LCG.seed(0)
sequence = [LCG.random() for i in range(10)]
print(sequence)

[1, 3, 7, 15, 31, 63, 127, 255, 255, 255]


## Forward/backward predictability  
 1. Attacker knowns that the **LCG** generated number 5. Is he able to find the internal state of the **LCG**? What is the problem?    
 2. Use LCG and generate next values - use appropriate seed. 
 3. Are you able to create "inverse" LCG f(x) = a*x+c mod m implies (f(x)-c)*a^-1 mod m 

In [62]:
LCG_inverse = PRNG(4, init_f=id, update_f = lambda x: lcg_update(x, a=2, c=1, m=256), output_f=id)

## RNG improvements - hash function
 1. Use MD5 and improve the generator **H** in a way that the internal state does not leak.  
 2. Is there any problem - how to change the **output_f** so it won't leak the internal state. 
 3. What is expected period of the generator? (Think about the birthday paradox.)
 4. What is the difference when we use AES in CTR mode as RNG? 
  - What is the period of the AES-CTR?
  - What is the state of the AES-CTR?
  - What is the seed of the AES-CTR?

In [63]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import hashlib 

def MD5(msg):
    h = hashlib.md5()
    h.update(msg)
    return h.digest()

H = PRNG(16, init_f=id, update_f = id, output_f=id)
H.random()
H.random()

def AES_CTR(msg, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CTR(iv))
    encryptor = cipher.encryptor() 
    ct = encryptor.update(msg) +  encryptor.finalize()
    return ct

# TRNG


 ## Entropy
 - The function **time_ns** produces certain amount of entropy per call.  
 - Implement function `entropy` that will produce required bits (`req_bits`) of entropy per call.
 - What is the appropriate way how to combine more values together? 
  - Can we use XOR (bitwise operator: ^) to combine random values? 
  - Or should we use hash function? 
  - What are pros and cons of XOR and hash?

In [57]:
def time_ns(state_size=8):
    return time.time_ns().to_bytes(state_size, byteorder='big') 

def XOR(bytes1, bytes2):
    return bytes(a ^ b for (a, b) in zip(bytes1, bytes2))

def entropy(req_bits):
    pass
    

## ANSIX931
  1. Implement ANSIX9.31 generator based on AES. 
  2. Use appropriate source of entropy (os.urandom(), secrets.token_bytes()) to reseed the generator. 

In [71]:
def AES_ECB(msg, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor() 
    ct = encryptor.update(msg) +  encryptor.finalize()
    return ct

import time
class ANSIX931: 
    def __init__(self, state_size=16, cipher=AES_ECB, key=b"\x00"*16) :
        self.state_size = state_size
        self.state = b"\x00"*state_size
        self.key = key
        
    def seed(self, seed):
        self.state = seed
        
    def random(self): 
        T = time.time_ns().to_bytes(self.state_size, byteorder='big') 
        # write update of state and compute the output 
        return T

RNG = ANSIX931()
RNG.random()

NameError: name 'R' is not defined